In [72]:
class EllipticCurve:
    def __init__(self, ec_a, ec_b, ec_p):
        # TODO Komprimierte Form einbauen result = a if a else b -> Nur mit Hex form arbeiten und unten die Tests überarbeiten
        self.ec_a = transformer(ec_a)
        self.ec_b = transformer(ec_b)
        self.ec_p = transformer(ec_p)
        '''
        ECC like y**2 = x**3 + ax + b
        :param x: x value of ECC in int or hex in str
        :param ec_a: steigung der ECC in int or hex in str
        :param ec_b: Const of ECC in int or hex in str
        :param ec_p: GF of ECC in int or hex in str
        '''
    def getY(self, ec_x):
        # TODO Vielleicht falsch impl nochmal drüber gucken
        """
        :param ec_x: X in komprimierter Form mit prefix 0x in str
        :return: y Value if exists
        """
        prefix, x = hexTransformer(ec_x)
        y2 =  pow(x,3,ec_p) + ec_a * x + ec_b
        if not isResidue(y2, self.ec_p):
            print("Punkt liegt nicht auf der Kurve")
            return None
        if y2 % 4 == 3:
            y =  pow(y2, modDivision(self.ec_p+1, 4, self.ec_p), ec_p)
            if prefix % 2 == 0 and y % 2 == 0:
                print(y)
                return y
            else:
                return (ec_p - y) % ec_p
        else:
            raise NotImplementedError("Tonelli-Shanks nicht implementiert bro")

    def point_add(self, P, Q):
        if P == "neutral" or Q == "neutral":
            return "neutral"
        x1, y1 = P
        x2, y2 = Q
        if x1  == x2 and (y1 + y2) % self.ec_p == 0 :
            return "neutral"
        if x1 == x2 and y1 == y2 :
           s = modDivision(3 * pow(x1,2)+ self.ec_a, 2 * y1, self.ec_p)
           x3 = (pow(s,2) - 2 * x1) % self.ec_p
           y3 = (s * (x1 -x3) -y1) % self.ec_p
        else:
            s = modDivision(y2-y1, x2-x1, self.ec_p)
            x3 = (pow(s,2) - x1 - x2) % self.ec_p
            y3 = (s * (x1-x3) -y1) % self.ec_p
        return x3, y3

    def point_sub(self, P, Q):
        x1, y1 = P
        x2, y2 = Q
        return self.point_add((x1,y1),(x2,-y2))





In [48]:
#ModArith
def modinv(ec_a, p):
        return pow(ec_a, -1, p)
def modDivision(x, y, p):
    return ((x % p) * pow(y, -1, p)) % p
def isResidue(z, p):
    # Euler Kriterium
    return (z ** modDivision(p-1, 2, p)) % p == 1
print(pow(1, modDivision(17+1, 4, 17), 17))


#Hilfsmethoden
def transformer(s):
    if type(s) == int:
        return s
    if s.startswith('0x'):
        return int(s, 16)

    raise Exception("Falscher Eingabewert")
def hexTransformer(s: str):
    if type(s) == int:
        return s
    if not s.startswith('0x'):
        print("Falscher Eingabewert")
        return None
    s = s.replace('0x', '')
    prefix = s[2:4]
    s = s[:2] + s[4:]
    return int(prefix), int(s,16)
print(int("0x224",16))
hexTransformer(0x224)
hexTransformer(0x03)


1
548
43690


In [80]:
x = hex(3131314)
print(type(x) == hex)
print(x)
ec_p = 31
ec_a = 16
ec_b = 7
E = EllipticCurve(ec_a, ec_b, ec_p)

P1 = (23,7)
P2 = (23,24)
Px = P1
for k in range(37):
    Px = E.point_add(Px, P1)
    if Px == P2:
        print(k+2)

# Testing Modulare Arithmetic
assert modDivision(5,7,31) == 14
assert isResidue(1, 17) == True

# Testing ECC
assert E.point_add((23,7),(23,24)) == "neutral"
assert E.point_add((2,4),(6,3)) == (25,25)
assert E.point_sub((2,4),(6,3)) == (28,26)
assert transformer(hex(313)) == 313
assert transformer("0x139") == 313
assert transformer(313) == 313
#E.transformer("Hallo")
t = (0o3, int("032ef3d968e2a32804c755414101e5f93de41bcdc099bfb4289fc940629385f0", 16))
assert hexTransformer("0x03032ef3d968e2a32804c755414101e5f93de41bcdc099bfb4289fc940629385f0") == (3, int("032ef3d968e2a32804c755414101e5f93de41bcdc099bfb4289fc940629385f0", 16))


# ECC gety
E = EllipticCurve(2,2,17)


False
0x2fc7b2
36
Punkt liegt nicht auf der Kurve


AssertionError: 

In [6]:
INF = (0,1,0)
class EllipticCurve:
    def __init__(self, ec_a, ec_b, ec_p):
        self.ec_a = ec_a
        self.ec_b = ec_b
        self.ec_p = ec_p

        '''
        ECC like y**2 = x**3 + ax + b
        :param x: x value of ECC in int or hex in str
        :param ec_a: steigung der ECC in int or hex in str
        :param ec_b: Const of ECC in int or hex in str
        :param ec_p: GF of ECC in int or hex in str
        '''

    def proj_add(self, P: (int,int,int), Q: (int,int,int)):
        if P == INF:
            return Q
        if Q == INF:
            return P

        X1, Y1, Z1 = P
        X2, Y2, Z2 = Q
        U1 = (Y2 * Z1) % self.ec_p
        U2 = (Y1 * Z2) % self.ec_p
        V1 = (X2 * Z1) % self.ec_p
        V2 = (X1 * Z2) % self.ec_p
        U = (U1 - U2) % self.ec_p
        V = (V1 - V2) % self.ec_p

        if V == 0:
            # check if Q is invers to P
            if U != 0:
                return INF
            # check if P is Q
            return self.proj_dbl(P)

        W = (Z1 * Z2) % self.ec_p
        A = (U**2 * W - V**3 - 2 * V**2 * V2) % self.ec_p
        X3 = (V * A) % self.ec_p
        Y3 = (U * (V**2 * V2 - A) - V**3 * U2) % self.ec_p
        Z3 = (V**3 * W) % self.ec_p
        return X3, Y3, Z3


    def proj_dbl(self, P: (int,int,int)):
        X, Y, Z = P
        W = (3 * X**2 + self.ec_a * Z**2) % self.ec_p
        S = (Y * Z) % self.ec_p
        B = (X * Y * S) % self.ec_p
        H = (W**2 - 8 * B) % self.ec_p
        X2 = (2 * H * S) % self.ec_p
        Y2 = (W * (4 * B - H) - 8 * Y**2 * S**2) % self.ec_p
        Z2 = (8 * S**3) % self.ec_p
        return X2, Y2, Z2


In [42]:
# Hilfsmethoden
def projtoaffin(P: (int,int,int), ec_p):
    X, Y, Z = P
    return modDivision(X, Z, ec_p), modDivision(Y, Z, ec_p), 1

In [58]:
ec_a = 0x7D5A0975FC2C3057EEF67530417AFFE7FB8055C126DC5C6CE94A4B44F330B5D9
ec_p = 0xA9FB57DBA1EEA9BC3E660A909D838D726E3BF623D52620282013481D1F6E5377
ec_b = 0x26DC5C6CE94A4B44F330B5D9BBD77CBF958416295CF7E1CE6BCCDC18FF8C07B6
E = EllipticCurve(ec_a, ec_b, ec_p)
P1 = (60306380415904663168568911239273826053144841234228559299517684417361346433053, 74653857005150983469598545140707432309023702960881435319026826228339031179596, 1)

P2 = E.proj_add(P1, P1)
Zinv = pow(Z3, -1, ec_p)
P_affin = projtoaffin(P2, ec_p)
P_affin

(74272161119054482720136981356270225780760229971611380324482072562025359940783,
 16405134000021918301262967552706457760831838374421496340393934000725840836545,
 1)